In [1]:
!pip install gym gym[atari]

  Using cached gym-0.18.3-py3-none-any.whl
  Using cached pyglet-1.5.15-py3-none-any.whl (1.1 MB)
  Using cached atari_py-0.2.9-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.8 MB)
You should consider upgrading via the '/usr/local/bin/python3.7 -m pip install --upgrade pip' command.


In [2]:
!pip install atari-py

You should consider upgrading via the '/usr/local/bin/python3.7 -m pip install --upgrade pip' command.


In [3]:
!python -m atari_py.import_roms './roms'

copying space_invaders.bin from ./roms/Space Invaders (1980) (Atari, Richard Maurer - Sears) (CX2632 - 49-75153) ~.bin to /usr/local/lib/python3.7/site-packages/atari_py/atari_roms/space_invaders.bin


In [1]:
import gym
import numpy as np
import random

In [2]:
env = gym.make('SpaceInvaders-ram-v0')

In [3]:
actions = env.action_space.n

In [4]:
env.observation_space.shape

(128,)

In [8]:
# episodes = 5
# for episode in range(1, episodes+1):
#     state = env.reset()
#     done = False
#     score = 0
    
#     while not done:
#         env.render()
#         action = random.choice([0,1,2,3,4,5])
#         state, reward, done, info = env.step(action)
#         score += reward
#     print(f'Episode: {episode} Score: {score}')
# env.close()

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten

In [8]:
!pip install keras-rl2

  Using cached keras_rl2-1.0.5-py3-none-any.whl (52 kB)
  Using cached tensorflow-2.5.0-cp37-cp37m-manylinux2010_x86_64.whl (454.3 MB)
  Using cached numpy-1.19.5-cp37-cp37m-manylinux2010_x86_64.whl (14.8 MB)
  Using cached keras_nightly-2.5.0.dev2021032900-py2.py3-none-any.whl (1.2 MB)
  Using cached flatbuffers-1.12-py2.py3-none-any.whl (15 kB)
  Using cached tensorflow_estimator-2.5.0-py2.py3-none-any.whl (462 kB)
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
  Using cached grpcio-1.34.1-cp37-cp37m-manylinux2014_x86_64.whl (4.0 MB)
  Using cached h5py-3.1.0-cp37-cp37m-manylinux1_x86_64.whl (4.0 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.18.5
    Uninstalling numpy-1.18.5:
      Successfully uninstalled numpy-1.18.5
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.37.1
    Uninstalling grpcio-1.37.1:
      Successfully uninstalled grpcio-1.37.1
  Attempting uninstall: tensorflow-estimator
    Found existing installation: t

In [6]:
from rl.agents import DQNAgent
from rl.memory import SequentialMemory, Memory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy, BoltzmannQPolicy, MaxBoltzmannQPolicy, BoltzmannGumbelQPolicy

In [7]:
def build_agent(model, actions, c, n):
    policy = BoltzmannGumbelQPolicy(c)
    memory = SequentialMemory(limit=1_000_000, window_length=3)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                   test_policy=BoltzmannQPolicy(1/(np.sqrt(n/actions))),
                   enable_dueling_network=True, dueling_type='avg', nb_actions=actions, 
                   nb_steps_warmup=10_000)
    return dqn

In [8]:
import pandas as pd

In [9]:
model = Sequential()
model.add(Flatten(input_shape=(3,) + env.observation_space.shape))
# model.add(Dense(128, activation='relu'))
# model.add(Dense(256, activation = 'relu'))
model.add(Dense(512, activation = 'relu'))
model.add(Dense(1024, activation = 'relu'))
model.add(Dense(512, activation = 'relu'))
model.add(Dense(256, activation = 'relu'))
model.add(Dense(128, activation = 'relu'))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(actions, activation='linear'))

In [10]:
dqn = build_agent(model, 6, .5, 10_000_000)

In [11]:
dqn.compile(optimizer='adam')

In [17]:
full_history = {
    'episode_reward':[],
    'nb_episode_steps':[],
    'nb_steps':[]
}

In [14]:
history_df = pd.read_csv('./data/history.csv')

In [12]:
dqn.load_weights('./saved_weights/dqn-bge.h5f')

In [ ]:
history = dqn.fit(env, nb_steps=500_000, visualize=False, verbose=1)
full_history['episode_reward']+=history.history['episode_reward']
full_history['nb_episode_steps']+=history.history['nb_episode_steps']
full_history['nb_steps']+=history.history['nb_steps']


Training for 500000 steps ...
Interval 1 (0 steps performed)
   98/10000 [..............................] - ETA: 15s - reward: 0.0000e+00

/usr/local/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:2426: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


10000/10000 [==============================] - 15s 2ms/step - reward: 0.2195
12 episodes - episode_reward: 181.667 [30.000, 485.000] - ale.lives: 2.090

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 160s 16ms/step - reward: 0.2070
15 episodes - episode_reward: 139.000 [15.000, 725.000] - loss: 7.656 - mean_q: 67.763 - ale.lives: 2.047

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 160s 16ms/step - reward: 0.2700
11 episodes - episode_reward: 223.636 [40.000, 410.000] - loss: 6.730 - mean_q: 65.980 - ale.lives: 1.972

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 162s 16ms/step - reward: 0.2910
11 episodes - episode_reward: 273.636 [30.000, 685.000] - loss: 6.938 - mean_q: 64.874 - ale.lives: 2.197

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 164s 16ms/step - reward: 0.2680
11 episodes - episode_reward: 253.182 [75.000, 505.000] - loss: 7.975 

In [16]:
df = pd.DataFrame.from_dict(full_history)

In [17]:
df.head()

,episode_reward,nb_episode_steps,nb_steps
0,170.0,701,701
1,180.0,842,1543
2,155.0,816,2359
3,75.0,658,3017
4,50.0,655,3672


In [18]:
history_df = history_df.append(df)

In [19]:
history_df.to_csv('./data/history.csv', index=False)

In [20]:
dqn.save_weights('./saved_weights/dqn-bge.h5f', overwrite=True)

In [13]:
scores = dqn.test(env, nb_episodes=5, visualize=True)
print(np.mean(scores.history['episode_reward']))
env.close()

Testing for 5 episodes ...


C:\Users\c24th\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\engine\training.py:2426: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


Episode 1: reward: 120.000, steps: 744
Episode 2: reward: 155.000, steps: 672
Episode 3: reward: 20.000, steps: 487
Episode 4: reward: 110.000, steps: 625
Episode 5: reward: 105.000, steps: 445
102.0


In [ ]:
!pip install pyopengl

In [17]:
history_df

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,episode_reward,nb_episode_steps,nb_steps
0,0,0.0,0.0,190.0,595,595
1,1,1.0,1.0,255.0,1225,1820
2,2,2.0,2.0,260.0,640,2460
3,3,3.0,3.0,290.0,656,3116
4,4,4.0,4.0,245.0,613,3729
...,...,...,...,...,...,...
1967,591,NaN,NaN,280.0,903,496316
1968,592,NaN,NaN,195.0,691,497007
1969,593,NaN,NaN,355.0,1168,498175
1970,594,NaN,NaN,315.0,904,499079


In [18]:
df = history_df

In [23]:
df.nb_steps = df.nb_episode_steps.cumsum()

In [24]:
df.tail()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,episode_reward,nb_episode_steps,nb_steps
1967,591,NaN,NaN,280.0,903,1655928
1968,592,NaN,NaN,195.0,691,1656619
1969,593,NaN,NaN,355.0,1168,1657787
1970,594,NaN,NaN,315.0,904,1658691
1971,595,NaN,NaN,120.0,762,1659453
